In [ ]:
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.summarystats as ds
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from delfi.simulator.Gauss import Gauss


n_params = 2
seed = 42
m = Gauss(dim=n_params, seed=seed)
p = dd.Uniform(lower=10*np.ones((n_params, )), 
               upper=20*np.ones(n_params, ),
               seed=seed)
s = ds.Identity()
g = dg.Default(model=m, prior=p, summary=s)

_, obs = g.gen(1)

# generate and plot
params, stats = g.gen(250)
plt.plot(stats, params, 'b.');
plt.xlabel('x')
plt.ylabel('theta')

# set up inference
#res = infer.CDELFI(g, obs=obs, n_components=2)

res = infer.Basic(g, n_hiddens=[10], seed=seed, svi=True)

# run with N samples
log, trn_data = res.run(1500)

In [ ]:
posterior = res.predict(np.array([15., 15.]).reshape(1,-1))

In [ ]:
posterior.xs[0].m

In [ ]:
posterior.xs[0].S

In [ ]:
plt.plot(log['loss'])

In [ ]:
from delfi.utils.io import load, save

In [ ]:
save(res, 'test.pkl')

In [ ]:
res2 = load('test.pkl')